# 🧈 Training MOCOv2 on CIFAR10 🔥 With pytorch lightning 🧈

In [1]:
import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl
import lightly

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import loggers as pl_loggers

SEED = 1

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/lightly/api/version_checking.py:57: Warning: You are using lightly version 1.1.18. There is a newer version of the package available. For compatability reasons, please upgrade your current version: pip install lightly==1.1.20
  warnings.warn(Warning(warning))


# ⬇️ Build Dataset

In [2]:
# DATA hyperparams
num_workers = 6
moco_batch_size = 512
classifier_train_batch_size = 512
classifier_test_batch_size = 512

In [3]:
# The dataset structure should be like this:
# cifar10/train/
#  L airplane/
#    L 10008_airplane.png
#    L ...
#  L automobile/
#  L bird/
#  L cat/
#  L deer/
#  L dog/
#  L frog/
#  L horse/
#  L ship/
#  L truck/
path_to_train = './data/cifar10_lightly/train/'
path_to_test = './data/cifar10_lightly/test/'

### Augmentations

In [4]:
################### Classifier Augmentations ###################
# Augmentations typically used to train on cifar-10
train_classifier_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])

# No additional augmentations for the test set
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((32, 32)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=lightly.data.collate.imagenet_normalize['mean'],
        std=lightly.data.collate.imagenet_normalize['std'],
    )
])

################### MOCO Augmentations ###################
# MoCo v2 uses SimCLR augmentations, additionally, disable blur
collate_fn = lightly.data.SimCLRCollateFunction(
    input_size=32,
    gaussian_blur=0.,
)

### Datasets

In [5]:
################### Classifier Datasets ###################
#Since we also train a linear classifier on the pre-trained moco model we
# reuse the test augmentations here (MoCo augmentations are very strong and
# usually reduce accuracy of models which are not used for contrastive learning.
# Our linear layer will be trained using cross entropy loss and labels provided
# by the dataset. Therefore we chose light augmentations.)
dataset_train_classifier = lightly.data.LightlyDataset(
    input_dir=path_to_train,
    transform=train_classifier_transforms
)

dataset_test = lightly.data.LightlyDataset(
    input_dir=path_to_test,
    transform=test_transforms
)

################### MOCO Dataset ###################
# We use the moco augmentations for training moco
dataset_train_moco = lightly.data.LightlyDataset(
    input_dir=path_to_train
)

### Dataloaders

In [6]:
################### Classifier Dataloaders ###################
dataloader_train_classifier = torch.utils.data.DataLoader(
    dataset_train_classifier,
    batch_size=classifier_train_batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
)

dataloader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=classifier_test_batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=num_workers
)

################### MOCO Dataloader ###################
dataloader_train_moco = torch.utils.data.DataLoader(
    dataset_train_moco,
    batch_size=moco_batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    drop_last=True,
    num_workers=num_workers
)

# 🧠 Load Model

In [7]:
# MODEL hyperparams
memory_bank_size = 4096
moco_max_epochs = 9000
downstream_max_epochs = 60
downstream_test_every = 100

In [8]:
import moco_model

In [9]:
model = moco_model.MocoModel(memory_bank_size, moco_max_epochs, 
                             downstream_max_epochs, dataloader_train_classifier, dataloader_test,
                            downstream_test_every=downstream_test_every)
#                             last_epoch=2989)

I will continue training from 900 epoch model

In [10]:
# # WHEN LOADING A SAVED MODEL DO IT LIKE DIS
model.load_from_checkpoint('./saved_models/resnet_moco/epoch=2989-train_loss_ssl=1.37.ckpt',
                          memory_bank_size=memory_bank_size, moco_max_epochs=moco_max_epochs,
                          downstream_max_epochs=downstream_max_epochs, dataloader_train_classifier=dataloader_train_classifier,
                           dataloader_test=dataloader_test,
                           downstream_test_every=downstream_test_every)
#                           last_epoch=2989);
# checkpoint_callback = ModelCheckpoint(dirpath="./saved_models/resnet_moco/epoch=897-train_loss_ssl=1.67.ckpt")

MocoModel(
  (resnet_moco): MoCo(
    (backbone): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (shortcut): Sequential()
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): SplitBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(

In [11]:
# you can also define a checkpoint callback to save best model like keras.
checkpoint_callback = ModelCheckpoint(
    dirpath='./saved_models/resnet_moco',
    filename='{epoch}-{train_loss_ssl:.2f}',
    save_top_k=5,
    verbose=True,
    monitor='train_loss_ssl',
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:446: UserWarning: Checkpoint directory ./saved_models/resnet_moco exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


# 🏋️‍♂️ Train

In [12]:
# use a GPU if available
gpus = 1 if torch.cuda.is_available() else 0
print(f'Using gpu: {bool(gpus)}')
if(gpus == 0): print('--- NOT USING GPUS THIS TAKE LONG TIME ---')

# set up tensorboard logger
tb_logger = pl_loggers.TensorBoardLogger(save_dir='./lightning_logs/', name=f'TESTmoco_{moco_max_epochs}eps')

Using gpu: True


In [13]:
# ### WHEN TRAINING FROM CHECKPOINT ###

# trainer = pl.Trainer(
#     resume_from_checkpoint="./saved_models/resnet_moco/epoch=2989-train_loss_ssl=1.37.ckpt",
#     max_epochs=moco_max_epochs,
#     gpus=gpus,
#     callbacks=[checkpoint_callback],
#     logger=tb_logger)
# trainer.fit(model, dataloader_train_moco)

In [ ]:
## WHEN TRAINING FROM SCRATCH (EPOCH 0) ###

trainer = pl.Trainer(max_epochs=moco_max_epochs, gpus=gpus, callbacks=[checkpoint_callback, lr_monitor], logger=tb_logger)
trainer.fit(
    model,
    dataloader_train_moco
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | resnet_moco | MoCo       | 23.0 M
1 | criterion   | NTXentLoss | 0     
-------------------------------------------
11.5 M    Trainable params
11.5 M    Non-trainable params
23.0 M    Total params
91.977    Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.classification.accuracy.Accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type       | Params
-------------------------------------------
0 | resnet_moco | MoCo       | 23.0 M
1 | fc          | Sequential | 267 K 
2 | accuracy    | Accuracy   | 0     
-------------------------------------------
267 K     Trainable params
23.0 M    Non-trainable params
23.3 M    Total params
93.048    Total estimated model params size (MB)


... training downstream classifier...


Validation sanity check: 0it [00:00, ?it/s]

...

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/shatz/miniconda3/envs/morebetter3/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1046: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Epoch 0, global step 96: train_loss_ssl reached 7.02350 (best 7.02350), saving model to "/home/shatz/Documents/more_better/saved_models/resnet_moco/epoch=0-train_loss_ssl=7.02.ckpt" as top 5


[2.0271335, 1.9899968]
[0.29552314, 0.28470102]
-----
